A transformation sequence from word beginWord to word endWord using a dictionary wordList is a sequence of words beginWord -> s1 -> s2 -> ... -> sk such that:

Every adjacent pair of words differs by a single letter.
Every si for 1 <= i <= k is in wordList. Note that beginWord does not need to be in wordList.
sk == endWord
Given two words, beginWord and endWord, and a dictionary wordList, return all the shortest transformation sequences from beginWord to endWord, or an empty list if no such sequence exists. Each sequence should be returned as a list of the words [beginWord, s1, s2, ..., sk].

Example 1:

Input: beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log","cog"]
Output: [["hit","hot","dot","dog","cog"],["hit","hot","lot","log","cog"]]
Explanation: There are 2 shortest transformation sequences:
"hit" -> "hot" -> "dot" -> "dog" -> "cog"
"hit" -> "hot" -> "lot" -> "log" -> "cog"
Example 2:

Input: beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log"]
Output: []
Explanation: The endWord "cog" is not in wordList, therefore there is no valid transformation sequence.
 

Constraints:

1 <= beginWord.length <= 5
endWord.length == beginWord.length
1 <= wordList.length <= 500
wordList[i].length == beginWord.length
beginWord, endWord, and wordList[i] consist of lowercase English letters.
beginWord != endWord
All the words in wordList are unique.
The sum of all shortest transformation sequences does not exceed 105.

In [106]:
from typing import List
from collections import defaultdict, deque
class Solution1:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        wordSet = set(wordList)
        
        # There is no transformation sequences if the endword doesn't exist in wordList
        if endWord not in wordSet:
            return []
        
        patternToWord = defaultdict(set)
        # Create a pattern-to-word dictionary 
        # It would be inefficient to find all the neighbors of each word
        for word in wordSet:
            for i in range(len(word)):
                pattern = word[:i] + "*" + word[i+1:]
                patternToWord[pattern].add(word)

        # BFS to find the shortest transformation sequences
        seen = {beginWord} # Keep track of visited words to avoid cycles
        queue = deque([(beginWord, [beginWord])]) # Initialize the queue with the starting word and its path
        res = [] # Store the final result
        currentLength = float('inf') # Store the length of the transformation sequences
        currentLevel = 1 # Store the level of nodes in BFS
        seenAtThisLevel = set()
        while queue:
            currentWord, currentPath = queue.popleft()
            
            # If the length of the current path exceeds the current shortest length, break the loop
            if len(currentPath) >= currentLength:
                break
            
            # Update seen at each level of BFS
            # This would have a TLE problem, because the neighbors in the next level can be
            # the next node of multiple nodes in the current level so we will have duplicates of nodes in the next level
            # In this case, we will have duplicates in each level
            if len(currentPath) > currentLevel:
                currentLevel = len(currentPath)
                seen = seen | seenAtThisLevel
                seenAtThisLevel = set()

            for i in range(len(currentWord)):
                # Generate a pattern for the current word
                pattern = currentWord[:i] + "*" + currentWord[i+1:]
                neighbors = patternToWord[pattern] # Get neighbors based on the pattern
                
                for neighbor in neighbors:
                    if neighbor == endWord:
                        res.append(currentPath + [neighbor]) # If the neighbor is the endWord, add the path to the result
                        currentLength = min(currentLength, len(currentPath + [neighbor]))
                    
                    elif neighbor not in seen:
                        seenAtThisLevel.add(neighbor)
                        queue.append((neighbor, currentPath + [neighbor])) # Add the neighbor and its path to the queue



        return res
# Optimize
# Moving Forward: start from begin
# Each level, find all connected nodes to the nodes of the current level in the record and add those to the record for the next level.
# Delete node from wordList to prevent revisiting and forming cycles
# Repeat the above steps until we reach end or we add no new nodes to the record for next level
# Moving Backward: start from end

# Do the same steps as moving forward but this time we will not record nodes but contruct our paths
# Construct paths in reversing order to have paths from begin to end
class Solution:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        wordSet = set(wordList) 

        # There is no transformation sequences if the endword doesn't exist in wordList
        if endWord not in wordSet:
            return []
        
        # Generate a dictionary to store patterns and neighbors
        # O(n*l)
        patternToWord = defaultdict(set)
        for word in wordSet:
            for i in range(len(word)):
                pattern = word[:i] + "*" + word[i+1:]
                patternToWord[pattern].add(word)

        queue = deque([(beginWord, 0)]) # Initialize the queue with the starting word annd its level
        seen = defaultdict(set)
        seen[0].add(beginWord) # Record the word of each level

        # Find all nodes until the endword
        found = False
        while queue:
            currentWord, wordLevel = queue.popleft()
            for i in range(len(currentWord)):
                pattern = currentWord[:i] + "*" + currentWord[i+1:]
                for neighbor in patternToWord[pattern]:
                    if neighbor in wordSet:
                        wordSet.remove(neighbor) # Keep track of the unvisited nodes to avoid circles
                        queue.append((neighbor, wordLevel + 1)) # Add all nodes in the sequences to deque
                        seen[wordLevel + 1].add(neighbor)

                    if neighbor == endWord:
                        endWordLevel = wordLevel + 1 # Get the endWord's level
                        found = True
                        break
                if found:
                    break
            if found:
                break

        if not found:
            return []
        
        #  Find the path from the endword to the beginword
        def isNeighbor(candidate: str, word: str) -> bool:
            for i in range(len(word)):
                pattern = word[:i] + "*" + word[i+1:]
                if candidate in patternToWord[pattern]:
                    return True
            return False
        
        res = [] # To store the transformation sequences

        def dfs(pathLevel, path: List[str]):
            print(pathLevel, path, seen[pathLevel])
            if pathLevel == 0:
                # All words of level 1 are the neighbors of beginword
                res.append([beginWord]+path)
                return
            
            for word in seen[pathLevel]:
                if isNeighbor(word, path[0]):
                    dfs(pathLevel - 1, [word] + path)

            return
        
        dfs(endWordLevel - 1, [endWord])
        return res
            
        


In [108]:
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log","cog"]
s = Solution()
s.findLadders(beginWord, endWord, wordList)


3 ['cog'] {'log', 'dog'}
2 ['log', 'cog'] {'dot', 'lot'}
1 ['lot', 'log', 'cog'] {'hot'}
0 ['hot', 'lot', 'log', 'cog'] {'hit'}
2 ['dog', 'cog'] {'dot', 'lot'}
1 ['dot', 'dog', 'cog'] {'hot'}
0 ['hot', 'dot', 'dog', 'cog'] {'hit'}


[['hit', 'hot', 'lot', 'log', 'cog'], ['hit', 'hot', 'dot', 'dog', 'cog']]